In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6532,application_1589299642358_1021,pyspark,idle,Link,Link,
6560,application_1589299642358_1049,pyspark,idle,Link,Link,
6568,application_1589299642358_1057,pyspark,idle,Link,Link,
6577,application_1589299642358_1066,pyspark,idle,Link,Link,
6582,application_1589299642358_1071,pyspark,busy,Link,Link,
6592,application_1589299642358_1081,pyspark,idle,Link,Link,
6593,application_1589299642358_1082,pyspark,idle,Link,Link,
6597,application_1589299642358_1086,pyspark,idle,Link,Link,
6598,application_1589299642358_1087,pyspark,busy,Link,Link,
6599,application_1589299642358_1088,pyspark,idle,Link,Link,


In [2]:
#

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6606,application_1589299642358_1095,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
transfers = spark.read.format('orc').load('/data/sbb/timetables/orc/transfers/000000_0')
transfers.show()

print('Transfer type value counts:')
transfers.rdd.map(lambda x: (x['transfer_type'], 1)).reduceByKey(lambda x, y: x+y).take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-------------+-----------------+
|from_stop_id|  to_stop_id|transfer_type|min_transfer_time|
+------------+------------+-------------+-----------------+
| 8500309:0:2| 8500309:0:4|            2|              180|
| 8500309:0:2| 8500309:0:5|            2|              180|
| 8500309:0:2| 8500309:0:3|            2|              180|
| 8500309:0:2| 8500309:0:1|            2|              180|
| 8500309:0:2|8500309:0:5B|            2|              180|
| 8500305:0:3| 8500305:0:1|            2|              180|
| 8500305:0:3| 8500305:0:2|            2|              180|
| 8500304:0:1| 8500304:0:2|            2|              120|
| 8500320:0:4| 8500320:0:2|            2|              120|
| 8500320:0:4| 8500320:0:3|            2|              120|
| 8500320:0:4| 8500320:0:5|            2|              120|
| 8500303:0:3| 8500303:0:2|            2|              120|
| 8500302:0:2| 8500302:0:3|            2|              120|
| 8500301:0:4| 8500301:0:3|            2

In [12]:
transfers = transfers.select('from_stop_id', 'to_stop_id', 'min_transfer_time')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
%%local
import pandas as pd
walking_times = pd.read_pickle('walking_times.pickle')

In [13]:
%%spark -o transfers -n 30000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12534
8503000:0:41/42 	8503000:0:43/44 	420.0 	8503000:0:41/42 	8503000:0:43/44 	6.762958 	0.135259
8503000:0:41/42 	8503000:0:14 	420.0 	8503000:0:41/42 	8503000:0:14 	13.525908 	0.270518
                                
                                NaN 	NaN 	NaN 	8596126 	8503000:0:12 	350.328370 	7.006567
NaN 	NaN 	NaN 	8596126 	8503000:0:13 	355.640800 	7.112816

In [36]:
%%local
merged = transfers.merge(walking_times, left_on=['from_stop_id', 'to_stop_id'], right_on=['source', 'target'], how='right')
mask_transfers = merged.to_stop_id.notnull() & merged.from_stop_id.notnull()
merged.loc[mask_transfers, 'walk_duration'] = merged.loc[mask_transfers, 'min_transfer_time'] / 60
merged.loc[~mask_transfers, 'walk_duration'] = merged.loc[~mask_transfers, 'duration'] + 2
display(merged[['source', 'target', 'walk_duration']])
merged[['source', 'target', 'walk_duration']].to_pickle('walking_edges.pickle')

Output()